# Gathering of Bayesian evidence for the ELM model

In [ ]:
import pickle
from pathlib import Path

import numpy as np
from matplotlib import pyplot as plt
from periodictable import elements
from tqdm import tqdm

In [ ]:
import elm

In [ ]:
import exfor_tools
import jitr

In [ ]:
# plotting
kd_color = "#bc5090"
qelm_color = "#f28d3d"
elm_color = "#4277c0"

In [ ]:
proton = (1, 1)
neutron = (1, 0)

In [ ]:
Ca40 = (40, 20)
Ca48 = (48, 20)

Zr90 = (90, 40)
Zr92 = (92, 40)
Zr94 = (94, 40)
Zr96 = (96, 40)

Sn124 = (124, 50)
Sn122 = (122, 50)
Sn120 = (120, 50)
Sn118 = (118, 50)
Sn116 = (116, 50)

Ba138 = (138, 56)

Ce140 = (140, 58)

Sm144 = (144, 62)
Sm148 = (148, 62)

Pb208 = (208, 82)
Pb206 = (206, 82)

In [ ]:
targets = [
    Ca40,
    Ca48,
    Zr90,
    Zr92,
    Zr94,
    Zr96,
    Sn116,
    Sn118,
    Sn120,
    Sn122,
    Sn124,
    Ba138,
    Ce140,
    Sm144,
    Sm148,
    Pb206,
    Pb208,
]

In [ ]:
core_solver = jitr.rmatrix.Solver(40)
channel_radius = 10 * np.pi
lmax = 20
theta_vis = np.linspace(0.1, np.pi, 180)

In [ ]:
def set_up_p_elastic_workspaces(target, entries_xs, entries_ratio):
    meas_ratio = exfor_tools.sort_measurements_by_energy(entries_ratio)
    meas_xs = exfor_tools.sort_measurements_by_energy(entries_xs)

    # only take non-ratio (p,p) data if it's not duplicate to ruth
    Eratio = [m.Elab for m in meas_ratio]
    Exs = [m.Elab for m in meas_xs]
    for i, E in enumerate(Exs):
        if E in Eratio:
            del Exs[i]
            del meas_xs[i]

    workspaces = []
    for m in meas_ratio:
        workspaces.append(
            elm.DifferentialXS(
                proton,
                target,
                m.Elab,
                m,
                theta_vis,
                core_solver,
                channel_radius,
                lmax,
                absolute_xs=False,
            )
        )
    for m in meas_xs:
        workspaces.append(
            elm.DifferentialXS(
                proton,
                target,
                m.Elab,
                m,
                theta_vis,
                core_solver,
                channel_radius,
                lmax,
                absolute_xs=True,
            )
        )

    workspaces.sort(key=lambda w: w.exp.Elab)

    return workspaces

In [ ]:
def set_up_n_elastic_workspaces(target, entries):
    measurements = exfor_tools.sort_measurements_by_energy(entries)

    return [
        elm.DifferentialXS(
            neutron, target, m.Elab, m, theta_vis, core_solver, channel_radius, lmax
        )
        for m in measurements
    ]

In [ ]:
workspaces_n_elastic = {}
workspaces_p_elastic = {}

for t in targets:
    A, Z = t
    n_data_path = Path(f"./{elements[Z]}{A}_n_elastic.pkl")
    if n_data_path.is_file():
        with open(n_data_path, "rb") as f:
            entries = pickle.load(f)
            n_entries = len(entries)
            n_measurements = int(
                np.sum([len(entry.measurements) for entry in entries.values()])
            )
        print(
            f"Read {n_measurements} measurements from {n_entries} entries from {n_data_path}"
        )
        ws = set_up_n_elastic_workspaces(t, entries)
        workspaces_n_elastic[t] = ws
    p_data_path = Path(f"./{elements[Z]}{A}_p_elastic.pkl")
    if p_data_path.is_file():
        with open(p_data_path, "rb") as f:
            entries = pickle.load(f)
            n_entries = len(entries["xs"]) + len(entries["ratio"])
            n_measurements = int(
                np.sum([len(entry.measurements) for entry in entries["xs"].values()])
                + np.sum(
                    [len(entry.measurements) for entry in entries["ratio"].values()]
                )
            )
        print(
            f"Read {n_measurements} measurements from {n_entries} entries from {p_data_path}"
        )
        ws = set_up_p_elastic_workspaces(t, entries["xs"], entries["ratio"])
        workspaces_p_elastic[t] = ws

In [ ]:
[ws.Elab for ws in workspaces_n_elastic[(40,20)]]